# Table of Contents
 <p><div class="lev1 toc-item"><a href="#Install-pcre-(not-pcre2)-and-swig" data-toc-modified-id="Install-pcre-(not-pcre2)-and-swig-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Install pcre (not pcre2) and swig</a></div><div class="lev1 toc-item"><a href="#Using-swig-to-create-python-modules-from-c-program" data-toc-modified-id="Using-swig-to-create-python-modules-from-c-program-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Using swig to create python modules from c program</a></div><div class="lev2 toc-item"><a href="#C-program-used-as-the-module" data-toc-modified-id="C-program-used-as-the-module-21"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>C program used as the module</a></div><div class="lev2 toc-item"><a href="#Interface-file-for-C-and-Python" data-toc-modified-id="Interface-file-for-C-and-Python-22"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Interface file for C and Python</a></div><div class="lev2 toc-item"><a href="#Compile-C-and-Interface-File-Using-Swig-and-Create-Python-Module-(.o,-.py,-and-_.so)" data-toc-modified-id="Compile-C-and-Interface-File-Using-Swig-and-Create-Python-Module-(.o,-.py,-and-_.so)-23"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Compile C and Interface File Using Swig and Create Python Module (.o, .py, and _.so)</a></div><div class="lev2 toc-item"><a href="#Makefile-To-Compile-Swig-Commands" data-toc-modified-id="Makefile-To-Compile-Swig-Commands-24"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>Makefile To Compile Swig Commands</a></div><div class="lev2 toc-item"><a href="#Python-Program-To-Test-the-Module" data-toc-modified-id="Python-Program-To-Test-the-Module-25"><span class="toc-item-num">2.5&nbsp;&nbsp;</span>Python Program To Test the Module</a></div>

# Install pcre (not pcre2) and swig

http://atmarkplant.com/install-swig-for-mac-os-x/  
http://www.pcre.org/  

First we need to install pcre then we can install swig in Mac. 

**Download**
download and unzip the these folders to ~/Applications/

**Install**    
sudo -H ./configure  
sudo -H make  
sudo -h make install  

**Check**  
pcre-config --version  
which swig  

** Add to Library PATH  in ~/.bash_profile**    
\##==============================================================================  
\# Path and Library for pcre (required for swig )  
\# swig is needed for C/C++ to python,java etc conversion  
##==============================================================================  
LD_LIBRARY_PATH=/usr/local/lib:/usr/lib  
export LD_LIBRARY_PATH  

# Using swig to create python modules from c program

https://en.wikibooks.org/wiki/Python_Programming/Extending_with_C

http://www.expobrain.net/2011/01/23/swig-tutorial-for-mac-os-x/
http://www.swig.org/Doc1.3/Python.html#Python_nn10
https://metaphor.ethz.ch/fsdb/sam/PythonTutorial/combining.html
https://stackoverflow.com/questions/145270/calling-c-c-from-python
http://intermediate-and-advanced-software-carpentry.readthedocs.io/en/latest/c++-wrapping.html

    
    

## C program used as the module

```c
/* File: hello.c */

// Ref: http://www.expobrain.net/2011/01/23/swig-tutorial-for-mac-os-x/

#include <stdio.h>
#include <time.h>
double My_variable = 3.0;

int fact(int n) {
    if (n <= 1) return 1;
    else return n*fact(n-1);
}

int my_mod(int x, int y) {
    return (x%y);
}

char *get_time()
{
    time_t ltime;
    time(&ltime);
    return ctime(&ltime);
}
```

## Interface file for C and Python 

```c
/*File: hello.i */
%module hello
%{
    /* Put header files here or function declarations like below */
    extern double My_variable;
    extern int fact(int n);
    extern int my_mod(int x, int y);
    extern char *get_time();
%}

extern double My_variable;
extern int fact(int n);
extern int my_mod(int x, int y);
extern char *get_time();
```

## Compile C and Interface File Using Swig and Create Python Module (.o, .py, and _.so)

 1. swig -python hello.i    (NOTE: python, not python3)  
>This creates three files: hello.py hello.o hello_wrap.c  
>hello.py is required by python in future.  
>hello.o and hello_wrap.c are intermediate files.  
    
2. gcc -O2  -fpic -c hello.c hello_wrap.c -I /Library/Frameworks/Python.framework/Versions/3.6/include/python3.6m
 
>This creates hello_wrap.o  
>This is intermediate file.  

3. gcc -O2  -L/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/config-3.6m-darwin -lpython3.6m -shared hello.o hello_wrap.o -o _hello.so

> This will create _hello.so.  
> This is required by future python program.  

## Makefile To Compile Swig Commands

In [15]:
makefile = """
# Program : Makefile for swig program to convert c to python3
# Author  : Bhishan Poudel
# Date    : Jun 30, 2017 Fri
#
#
#
# Convension: There should be foo.c and foo.i and module name should be foo.
# Python3 include path: python3-config --includes
# Python3 librarty path: python3-config --ldflags
#
# Description:
# 1. swig -python hello.i    (NOTE: python, not python3)
#    This creates three files: hello.py hello.o hello_wrap.c
#    hello.py is required by python in future.
#    hello.o and hello_wrap.c are intermediate files.
#
# 2. gcc -O2  -fpic -c hello.c hello_wrap.c -I /Library/Frameworks/Python.framework/Versions/3.6/include/python3.6m
#
#    This creates hello_wrap.o
#    This is intermediate file.
#
# 3. gcc -O2  -L/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/config-3.6m-darwin -lpython3.6m -shared hello.o hello_wrap.o -o _hello.so
#
#   This will create _hello.so.
#   This is required by future python program.
# 
# Note: python needs three files: hello.py hello.o and _hello.so
#       hello.c and hello.c are created by user. 
#
#       For python2 use appropriate INCLUDE and LIB directory.
#
# TO INSTALL SWIG:
# First install pcre-8.39, then install swig usig configure,make,makefile. 

CC=gcc -O2 
INC=-I /Library/Frameworks/Python.framework/Versions/3.6/include/python3.6m
LIB=-L/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/config-3.6m-darwin -lpython3.6m -shared
PRG=hello


# Execute program and clean it
default: all


all : swig_ wrap_ build_ clean_
swig_ :
	swig -python $(PRG).i

wrap_ :
	$(CC) -fpic -c $(PRG).c $(PRG)_wrap.c $(INC)

build_ :
	$(CC) $(LIB) $(PRG).o $(PRG)_wrap.o -o _$(PRG).so

clean_ :
	rm -f $(PRG).pyc $(PRG)_wrap.c $(PRG)_wrap.o
	rm -rf __pycache__ .DS_Store



# Utility targets
.PHONY: clean

clean:
	rm -f $(PRG).pyc $(PRG)_wrap.c $(PRG)_wrap.o
	rm -rf __pycache__ .DS_Store
	rm -f $(PRG).py $(PRG).py _$(PRG).so




#swig -python hello.i
#gcc -O2 -fpic -c hello.c hello_wrap.c -I/Library/Frameworks/Python.framework/Versions/3.6/include/python3.6m 

#gcc -O2 -L/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/config-3.6m-darwin -lpython3.6m -shared hello.o hello_wrap.o -o _hello.so
#clear; python3 hello_test.py
#rm -f hello_wrap.c hello_wrap.o hello.pyc
""";

## Python Program To Test the Module

```python
# File: hello_test.py
# It needs three files hello.py, hello.o and _hello.so
# These three are created by swig using hello.c and hello.i
# swig = Simplified Wrapper and Interface Generator
# i is the interface file.

import hello

a = hello.fact(5)
print(('The value of a = ', a))

```